In [120]:
import os, glob, math
import numpy as np
from scipy.spatial import cKDTree
from PIL import Image


NPY_FOLDER   = r"C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\processed"
OUT_FOLDER   = r"C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\panos"
IMG_WIDTH    = 64
IMG_HEIGHT   = 32
MAX_DIST     = 10.0
RAY_COUNT    = 2048

os.makedirs(OUT_FOLDER, exist_ok=True)


## process depth maps

In [121]:

# --------------------------------------------------------
# 1. Fibonacci Sphere Directions (used in Rhino as well)
# --------------------------------------------------------
def fibonacci_sphere_dirs(n: int) -> np.ndarray:
    """Generate `n` unit vectors evenly distributed on a sphere using the Fibonacci method.
       Y is treated as the vertical axis (elevation).
    Returns:
        dirs: (n, 3) numpy array of unit direction vectors (x, y, z)
    """
    dirs = np.zeros((n, 3), dtype=np.float64)
    offset = 2.0 / n
    inc = math.pi * (3.0 - math.sqrt(5.0))  # golden angle

    for i in range(n):
        y = ((i * offset) - 1.0) + (offset / 2.0)  # elevation (Y-axis)
        r = math.sqrt(max(0.0, 1.0 - y * y))
        phi = i * inc
        x = math.cos(phi) * r
        z = math.sin(phi) * r
        dirs[i, :] = (x, y, z)
    
    return dirs  # shape: (n, 3)


# --------------------------------------------------------
# 2. Convert Directions → (Azimuth, Elevation)
# --------------------------------------------------------
def dirs_to_angles_xyz(dirs: np.ndarray) -> tuple:
    """Convert Cartesian directions (x, y, z) to azimuth and elevation angles.
    Returns:
        azimuth: (n,) in [-π, π]
        elevation: (n,) in [-π/2, π/2]
    """
    x = dirs[:, 0]
    y = dirs[:, 1]
    z = dirs[:, 2]
    az = np.arctan2(z, x)              # azimuth (longitude)
    el = np.arcsin(np.clip(y, -1.0, 1.0))  # elevation (latitude)
    return az, el


# --------------------------------------------------------
# 3. Map Angles to Pixel Coordinates
# --------------------------------------------------------
def angles_to_pixels(az: np.ndarray, el: np.ndarray, W: int, H: int) -> tuple:
    """Map azimuth and elevation to pixel coordinates in an equirectangular panorama.
    Returns:
        u, v: fractional pixel positions in [0, W) and [0, H)
    """
    u = ((az + np.pi) / (2.0 * np.pi)) * (W - 1)
    v = ((np.pi / 2.0 - el) / np.pi) * (H - 1)
    return u, v


In [122]:
def rasterize_frame_log(distances, u, v, W, H, max_dist=10.0):
    eps = 1e-3
    clipped = np.clip(distances, eps, max_dist)
    log_d = np.log(clipped)

    pano = np.full((H, W), np.inf)
    ui = np.clip(np.round(u).astype(int), 0, W - 1)
    vi = np.clip(np.round(v).astype(int), 0, H - 1)
    lin_idx = vi * W + ui
    flat = pano.flatten()
    np.minimum.at(flat, lin_idx, log_d)
    pano = flat.reshape(H, W)
    pano[np.isinf(pano)] = np.nan

    known_mask = np.isfinite(pano)
    if known_mask.any():
        ys, xs = np.where(known_mask)
        vals = pano[ys, xs]
        grid_y, grid_x = np.mgrid[0:H, 0:W]
        grid_points = np.column_stack([grid_y.ravel(), grid_x.ravel()])
        tree = cKDTree(np.column_stack([ys, xs]))
        _, nn_idx = tree.query(grid_points, k=1)
        filled = vals[nn_idx].reshape(H, W)
        log_depth = np.where(np.isfinite(filled), filled, np.log(max_dist))
    else:
        log_depth = np.full((H, W), np.log(max_dist))

    lo, hi = np.nanpercentile(log_depth, [1, 99])
    norm = (log_depth - lo) / (hi - lo)
    img = np.clip(norm * 255.0, 0, 255).astype(np.uint8)
    return img


## batch convert

In [123]:
def window_to_panos(npy_path, out_folder, W=512, H=256, max_dist=10.0):
    arr = np.load(npy_path)  # shape: (T, 2048), T<=7
    if arr.ndim != 2 or arr.shape[1] != RAY_COUNT:
        raise ValueError(f"Unexpected shape for {npy_path}: {arr.shape} (expected T x {RAY_COUNT})")

    # Precompute directions and pixel coordinates once
    dirs = fibonacci_sphere_dirs(RAY_COUNT)
    az, el = dirs_to_angles_xyz(dirs)
    u, v = angles_to_pixels(az, el, W, H)

    base = os.path.splitext(os.path.basename(npy_path))[0]
    out_paths = []

    T = arr.shape[0]
    for t in range(T):
        distances = arr[t]  # (2048,)
        img = rasterize_frame_log(distances, u, v, W, H)
        out_name = f"{base}_f{t:02d}.png"
        out_path = os.path.join(out_folder, out_name)
        Image.fromarray(img).save(out_path)
        out_paths.append(out_path)
    return out_paths

def batch_convert(npy_folder=NPY_FOLDER, out_folder=OUT_FOLDER, W=IMG_WIDTH, H=IMG_HEIGHT, max_dist=MAX_DIST):
    files = sorted(glob.glob(os.path.join(npy_folder, "*.npy")))
    total = 0
    for f in files:
        outs = window_to_panos(f, out_folder, W=W, H=H, max_dist=max_dist)
        total += len(outs)
        print(f"[OK] {f} → {len(outs)} images")
    print(f"Done. Wrote {total} images to {out_folder}")

if __name__ == "__main__":
    batch_convert()


[OK] C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\processed\t1-1_curve_01_peak_011.npy → 7 images
[OK] C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\processed\t1-1_curve_02_peak_011.npy → 7 images
[OK] C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\processed\t1-1_curve_03_peak_011.npy → 7 images
[OK] C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\processed\t1-1_curve_04_peak_011.npy → 7 images
[OK] C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\processed\t1-1_curve_05_peak_011.npy → 7 images
[OK] C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\processed\t1-1_curve_06_peak_011.npy → 7 images
[OK] C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\processed\t1-1_curve_07_peak_011.npy → 7 images
[OK] C:\Users\shrua\OneDrive\Desktop\threshold project\threshold\data\processed\t1-1_curve_08_peak_011.npy → 7 images
[OK] C:\Users\shrua\OneDrive\Desktop\threshold project\t